### Imports

In [1]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import joblib

### Model

In [2]:
class GNN(nn.Module):
    def __init__(self, in_dim=4, hidden=16, out_dim=2):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(in_dim, hidden),
            nn.ReLU(),
            nn.Linear(hidden, out_dim)
        )
    def forward(self, x):
        return self.net(x)

### Load Model

In [3]:
model = GNN()
model.load_state_dict(torch.load("../models/gnn_model.pth", weights_only=True))
model.eval()

scaler = joblib.load("../models/scaler_model.pkl")

### Inputs

In [4]:
edges_input_df = pd.read_csv("../inputs/edges_input.csv")
edges_input = edges_input_df.values.tolist()

X_test = torch.tensor(scaler.transform(edges_input), dtype=torch.float)

### Inference

In [5]:
with torch.no_grad():
    logits = model(X_test)
    probs = F.softmax(logits, dim=1)
    preds = probs.argmax(dim=1)

CLASS_NAMES = {0: "Ruim", 1: "Bom"}

In [6]:
print(f"{'RSSI':<6} {'ETX':<5} {'Delay':<6} {'Busy':<6} {'Classe':<8} {'Status':<8} {'Confiança':<10}")
print(f"{'-'*55}")

for i, row in enumerate(edges_input):
    cls = preds[i].item()
    conf = probs[i, cls].item()
    print(f"{row[0]:<6.0f} {row[1]:<5.1f} {row[2]:<6.0f} {row[3]:<6.2f} "
          f"{cls:<8} {CLASS_NAMES[cls]:<8} {conf:<10.2f}")

RSSI   ETX   Delay  Busy   Classe   Status   Confiança 
-------------------------------------------------------
-55    2.0   17     0.30   1        Bom      0.99      
-85    3.8   30     0.10   1        Bom      0.71      
-35    1.2   5      0.05   1        Bom      1.00      
-88    9.5   95     0.90   0        Ruim     1.00      
-40    8.2   85     0.80   0        Ruim     1.00      
-87    1.5   8      0.15   1        Bom      0.96      
-62    2.8   32     0.28   1        Bom      0.95      
-58    3.2   28     0.32   1        Bom      0.94      
-52    2.1   20     0.25   1        Bom      0.99      
-48    2.5   25     0.22   1        Bom      0.99      
-50    1.8   22     0.28   1        Bom      0.99      
-75    5.2   65     0.60   0        Ruim     1.00      
-78    4.8   70     0.65   0        Ruim     1.00      
-72    5.5   60     0.58   0        Ruim     1.00      
-45    4.5   15     0.70   0        Ruim     0.52      
-70    1.9   45     0.20   1        Bom      0.9

### Save

In [7]:
inference_output = pd.DataFrame({
    'class': preds.numpy(),
    'confidence': torch.max(probs, dim=1)[0].numpy()
})

os.makedirs("../outputs", exist_ok=True)
inference_output.to_csv("../outputs/inference_output.csv", index=False)

print("Inferência salva em ../outputs/inference_output.csv")

Inferência salva em ../outputs/inference_output.csv
